In [17]:
import tensorflow as tf
import tensorflowjs as tfjs
import pandas as pd
import numpy as np
import json

In [18]:
trainsplit = 7000
emotionDict = {'joy': 0, 'fear': 1, 'anger': 2, 'sadness': 3, 'disgust': 4, 'shame': 5, 'guilt': 6}

dataFrame = pd.read_csv('./ISEAR.csv')
for i in range(len(dataFrame['emotion'])):
    dataFrame['emotion'][i] = emotionDict[dataFrame['emotion'][i]]
xTrain = dataFrame['sentence'][:trainsplit]
yTrain = dataFrame['emotion'][:trainsplit]
xTest = dataFrame['sentence'][trainsplit:]
yTest = dataFrame['emotion'][:trainsplit]
dataFrame.tail()

,emotion,sentence
7511,5,Two years back someone invited me to be the tu...
7512,5,I had taken the responsibility to do something...
7513,1,I was at home and I heard a loud sound of spit...
7514,6,I did not do the homework that the teacher had...
7515,1,I had shouted at my younger brother and he was...


In [61]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(xTrain)
vocab = tokenizer.word_index
vocabSize = len(vocab)
f = open('../src/stokenizer.json', 'w')
f.write(json.dumps(vocab))
f.close()

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [20]:
xTrain = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataFrame['sentence'][:trainsplit]))
yTrain = tf.keras.utils.to_categorical((dataFrame['emotion'][:trainsplit]))
xTest = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences(dataFrame['sentence'][trainsplit:]))
yTest = tf.keras.utils.to_categorical((dataFrame['emotion'][trainsplit:]))
xTrain.shape, yTrain.shape, vocabSize

((7000, 167), (7000, 7), 8796)

In [52]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 64, input_length=167))
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)))
model.add(tf.keras.layers.Dense(64, activation='relu'))
model.add(tf.keras.layers.Dense(7, activation='softmax'))

model.compile(loss='categorical_crossentropy' , optimizer='adam', metrics=['accuracy'])

In [53]:
model.fit(xTrain, yTrain, epochs=15)

Epoch 1/15
219/219 [==============================] - 9s 23ms/step - loss: 1.7926 - accuracy: 0.2729
Epoch 2/15
219/219 [==============================] - 5s 22ms/step - loss: 1.1645 - accuracy: 0.5780
Epoch 3/15
219/219 [==============================] - 6s 26ms/step - loss: 0.7774 - accuracy: 0.7399
Epoch 4/15
219/219 [==============================] - 7s 30ms/step - loss: 0.5135 - accuracy: 0.8329
Epoch 5/15
219/219 [==============================] - 6s 28ms/step - loss: 0.3726 - accuracy: 0.8830
Epoch 6/15
219/219 [==============================] - 7s 30ms/step - loss: 0.2580 - accuracy: 0.9216
Epoch 7/15
219/219 [==============================] - 8s 35ms/step - loss: 0.1900 - accuracy: 0.9394
Epoch 8/15
219/219 [==============================] - 6s 26ms/step - loss: 0.1626 - accuracy: 0.9509
Epoch 9/15
219/219 [==============================] - 5s 22ms/step - loss: 0.1329 - accuracy: 0.9611
Epoch 10/15
219/219 [==============================] - 4s 20ms/step - loss: 0.0983 - accura

In [54]:
model.evaluate(xTest, yTest)

17/17 [==============================] - 1s 23ms/step - loss: 3.1705 - accuracy: 0.5484


[3.1704599857330322, 0.5484496355056763]

In [60]:
inpt = input('enter the text')
text = tf.keras.preprocessing.sequence.pad_sequences(tokenizer.texts_to_sequences([inpt]), padding='pre', maxlen=66)
dictEmotion = {0: 'joy', 1: 'fear', 2: 'anger', 3: 'sadness', 4: 'disgust', 5: 'shame' , 6: 'guilt'}

print(text, model.predict(text), dictEmotion[np.argmax(model.predict(text))])

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    2  442    4 2420]] [[9.9966991e-01 2.2148837e-08 9.3392131e-08 1.4732043e-05 3.4667814e-08
  3.1526538e-04 2.6533411e-09]] joy


In [59]:
tfjs.converters.save_keras_model(model, '../public/model')